Importations des fonctions

In [2]:
import requests
import time
import pandas as pd

Variables liées à l'API de Riot

In [3]:
api_key = "RGAPI-22dd4b79-8878-4453-8104-a5f363fa0da6"
region = "euw1"

Les programmes suivant visent à retrouver les puuid des joueurs qui sont des identifiants uniques

Fonction prenant une page de joueurs (205 joueurs) selon leur Rang, on isole leur ID des informations que l'on récupère que l'on stock dans un fichier csv.
Cet ID nous permettra ensuite de récupérer le puuid que l'on cherche.

In [4]:
def summ_ID_puller(division, tier, page):

    url_pull = "https://{}.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{}/{}?page={}&api_key={}".format(
        region, division, tier, page, api_key)
    profile_list = requests.get(url_pull).json()
    num_profiles = len(profile_list)
    summID_list = []

    for profile in range(0, num_profiles):
        summID_list.append(profile_list[profile]['summonerId'])

    df = pd.DataFrame(summID_list, columns=["Summoner ID"])
    df.to_csv('summID.csv', mode='a')

Boucle appliquant la fonction précédante pour 20 pages d'IDs et pour tous les joueurs Diamants

In [5]:
for tier in ["I", "II", "III", "IV"]:
    for page in range(1, 20):
        time.sleep(1)
        summ_ID_puller("DIAMOND", tier, page)

KeyboardInterrupt: 

Programme visant à stocker les puuid des joueurs (identifiant unique permettant de retrouver l'historique des joueurs plus tard)

Préparation des variables utiles

In [12]:
summoner_IDs = pd.read_csv("summID.csv")
accountID_list = []
summID_list = summoner_IDs["Summoner ID"]

Fonction récupérant l'puuid d'un joueur à partir de son ID

In [8]:
def acct_ID_puller(summID):
    url_acct_pull = "https://{}.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}".format(
        region, summID, api_key)
    account_info = requests.get(url_acct_pull).json()
    accountID_list.append(account_info["puuid"])

On applique la fonction précedente sur notre liste d'IDs de joueurs afin de récupérer leur puuid.

In [13]:
for summID_idx in range(0, 10):
    time.sleep(1)
    if summID_list[summID_idx] == "Summoner ID":
        pass
    else:
        try:
            acct_ID_puller(summID_list[summID_idx])
        except KeyError:
            print("keyerror")

On transforme tout ça en DataFrame et on le stock séparément dans un fichier csv.

In [14]:
df = pd.DataFrame(accountID_list, columns=["AccountPUUID"])
df.to_csv('accountPUUId.csv', mode='x')